# Postgresql setup and basics

In [2]:
# SSH into colab instance for prerequisite setups
# ngrok only allows one instance at a time

#Generate root password
import random, string
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip
#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

#Ask token
print("Copy authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')
#Print root password
print("Root password: {}".format(password))
#Get public address
! curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Copy authtoken from https://dashboard.ngrok.com/auth
··········
Root password: zW8Iib8RHZfeovR7DFce
tcp://0.tcp.ngrok.io:10450


In [0]:
# setup postgresql in colab terminal
# https://itsfoss.com/install-postgresql-ubuntu/

# sudo apt install postgresql
# service postgresql start

# In psql, create a user 'student' with password 'student', 
# and create a database 'student'.

In [0]:
import psycopg2

# Connect to the postgresql database
try:
  conn = psycopg2.connect("host=127.0.0.1 dbname=student user=student password=student")
except psycopg2.Error as e:
  print(e)

In [0]:
# Get cursor
try:
  cur = conn.cursor()
except:
  print("Error getting cursor")
  print(e)

In [0]:
# set autocommit to true
conn.set_session(autocommit=True)

In [0]:
try:
  cur.execute("CREATE DATABASE test")
except psycopg2.Error as e:
  print("Could not create db")

In [0]:
# Close connection and reconnect to the 'test' database
conn.close()

try:
  conn = psycopg2.connect("host=127.0.0.1 dbname=test user=student password=student")
except psycopg2.Error as e:
  print("Could not connect")

try:
  cur = conn.cursor()
except psycopg2.Error as e:
  print("Could not get cursor")

conn.set_session(autocommit=True)

Here we are trying to model a table of music albums, consisting of album names, artist names and release year.

In [0]:
try:
  cur.execute("CREATE TABLE IF NOT EXISTS music_lib (album_name varchar, \
                                                     artist_name varchar, \
                                                     year int);")
except psycopg2.Error as e:
  print("Table creation failed")

In [16]:
# sanity check
try:
  cur.execute("SELECT COUNT(*) FROM music_lib")
except psycopg2.Error as e:
  print("Table creation failed")

print(cur.fetchall())

[(0,)]


The 0 return val shows that table has indeed been created.


In [0]:
try:
  cur.execute("INSERT INTO music_lib (album_name, artist_name, year) \
               VALUES (%s, %s, %s)", \
               ("Musicality", "Bundles of Joi", 2019))
except psycopg2.Error as e:
  print("Row insertion failed")

try:
  cur.execute("INSERT INTO music_lib (album_name, artist_name, year) \
               VALUES (%s, %s, %s)", \
               ("Paradise", "Coldplay", 2012))
except psycopg2.Error as e:
  print("Row insertion failed")

In [19]:
try:
  cur.execute("SELECT * FROM music_lib;")
  row = cur.fetchone()
  while row:
    print(row)
    row = cur.fetchone()
except psycopg2.Error as e:
  print("Error reading table")

('Musicality', 'Bundles of Joi', 2019)
('Paradise', 'Coldplay', 2012)


In [0]:
# Drop table and close the connection
cur.execute("DROP TABLE music_lib")

cur.close()
conn.close()